In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
dataset = load_dataset('syntaxnoob/weather-prediction-prototype-aws')
train_dataset = dataset['train']
df = train_dataset.to_pandas()
df.to_csv('weather_data.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/284024 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/31586 [00:00<?, ? examples/s]

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=95d73adc16e380b9395a1372e66af7289bd382d2986f136b05841fe10559104b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Weather Data Loading") \
    .getOrCreate()
df = spark.read.csv('weather_data.csv', header=True, inferSchema=True)
df.show()

+-------------------+------+------------------+------------------+-------------------+---------------+------------------+------------------+-------------------+-------------------+-------------------+------+----------+----------+------------------+------------------+------------------+------------------+---------------------+---------------------+-------------------+-----------------------+-----------------------+---------------+-------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|        timestamp_1|code_1|    air_pressure_1| air_temperature_1|relative_humidity_1|precipitation_1|      wind_speed_1|  wind_direction_1|        timestamp_2|    tar_timestamp_1|    tar_timestamp_2|code_2|tar_code_1|tar_code_2|    air_pressure_2|tar_air_pressure_1|tar_air_pressure_2| air_temperature_2|tar_air_temperature_1|tar_air_temperature_2|relative_humidity_2|tar_relative_humidity_1|tar_relative_hu

# Handling missing values:

In [ ]:
from pyspark.sql.functions import col, count,when
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+------+--------------+-----------------+-------------------+---------------+------------+----------------+-----------+---------------+---------------+------+----------+----------+--------------+------------------+------------------+-----------------+---------------------+---------------------+-------------------+-----------------------+-----------------------+---------------+-------------------+-------------------+------------+----------------+----------------+----------------+--------------------+--------------------+
|timestamp_1|code_1|air_pressure_1|air_temperature_1|relative_humidity_1|precipitation_1|wind_speed_1|wind_direction_1|timestamp_2|tar_timestamp_1|tar_timestamp_2|code_2|tar_code_1|tar_code_2|air_pressure_2|tar_air_pressure_1|tar_air_pressure_2|air_temperature_2|tar_air_temperature_1|tar_air_temperature_2|relative_humidity_2|tar_relative_humidity_1|tar_relative_humidity_2|precipitation_2|tar_precipitation_1|tar_precipitation_2|wind_speed_2|tar_wind_speed_1|t

In [ ]:
df.printSchema()

root
 |-- timestamp_1: timestamp (nullable = true)
 |-- code_1: integer (nullable = true)
 |-- air_pressure_1: double (nullable = true)
 |-- air_temperature_1: double (nullable = true)
 |-- relative_humidity_1: double (nullable = true)
 |-- precipitation_1: double (nullable = true)
 |-- wind_speed_1: double (nullable = true)
 |-- wind_direction_1: double (nullable = true)
 |-- timestamp_2: timestamp (nullable = true)
 |-- tar_timestamp_1: timestamp (nullable = true)
 |-- tar_timestamp_2: timestamp (nullable = true)
 |-- code_2: double (nullable = true)
 |-- tar_code_1: double (nullable = true)
 |-- tar_code_2: double (nullable = true)
 |-- air_pressure_2: double (nullable = true)
 |-- tar_air_pressure_1: double (nullable = true)
 |-- tar_air_pressure_2: double (nullable = true)
 |-- air_temperature_2: double (nullable = true)
 |-- tar_air_temperature_1: double (nullable = true)
 |-- tar_air_temperature_2: double (nullable = true)
 |-- relative_humidity_2: double (nullable = true)
 |-- 

Random forest

In [ ]:
# Selecting numerical columns for clustering
features = [col for col in df.columns if not col.startswith('tar_') and 'timestamp' not in col and 'code' not in col]

In [ ]:
#Converting 'timestamp_1' to more useful features and extending feature set for effective use of timestamp in modelling
from pyspark.sql.functions import hour, dayofweek, month
df = df.withColumn('hour_1', hour('timestamp_1'))
df = df.withColumn('day_of_week_1', dayofweek('timestamp_1'))
df = df.withColumn('month_1', month('timestamp_1'))
features.extend(['hour_1', 'day_of_week_1', 'month_1'])

In [ ]:
#Converting 'timestamp_2' in a similar way
df = df.withColumn('hour_2', hour('timestamp_2'))
df = df.withColumn('day_of_week_2', dayofweek('timestamp_2'))
df = df.withColumn('month_2', month('timestamp_2'))
features.extend(['hour_2', 'day_of_week_2', 'month_2'])

In [ ]:
df[features]

DataFrame[air_pressure_1: double, air_temperature_1: double, relative_humidity_1: double, precipitation_1: double, wind_speed_1: double, wind_direction_1: double, air_pressure_2: double, air_temperature_2: double, relative_humidity_2: double, precipitation_2: double, wind_speed_2: double, wind_direction_2: double, hour_1: int, day_of_week_1: int, month_1: int, hour_1: int, day_of_week_1: int, month_1: int, hour_2: int, day_of_week_2: int, month_2: int]

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
#Main function of Random Forest Regressor
feature_columns = [
    'code_1', 'air_temperature_1', 'relative_humidity_1', 'precipitation_1',
    'wind_speed_1', 'wind_direction_1', 'air_pressure_2', 'air_temperature_2',
    'relative_humidity_2', 'precipitation_2', 'wind_speed_2', 'wind_direction_2',
    'hour_2', 'day_of_week_2', 'month_2', 'hour_1', 'day_of_week_1', 'month_1'
]


targets = [
    'air_pressure_1', 'air_temperature_1', 'relative_humidity_1', 'precipitation_1',
    'wind_speed_1', 'wind_direction_1', 'air_pressure_2', 'air_temperature_2',
    'relative_humidity_2', 'precipitation_2', 'wind_speed_2', 'wind_direction_2'
]

def train_and_evaluate_model(df, feature_columns, target):
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    rf = RandomForestRegressor(featuresCol="features", labelCol=target, numTrees=50, maxDepth=10)
    pipeline = Pipeline(stages=[assembler, rf])
    train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)
    model = pipeline.fit(train_data)
    predictions = model.transform(test_data)

    # Evaluating the model using the RMSE metric
    evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE) for {target} on test data = {rmse}")
    predictions.select(target, "prediction").show(5)

    return model, rmse

# Training and evaluate a model for each target variable
models = {}
for target in targets:
    model, rmse = train_and_evaluate_model(df, feature_columns, target)
    models[target] = model

Root Mean Squared Error (RMSE) for air_pressure_1 on test data = 0.01777192079233687
+------------------+-------------------+
|    air_pressure_1|         prediction|
+------------------+-------------------+
|0.4756533700137549|0.46430077051568447|
|0.7921137093076565| 0.7833656339295042|
|0.4767537826685005| 0.4664177671755607|
|0.7974323704722592| 0.7852340656125913|
|0.4767537826685005| 0.4914772614592113|
+------------------+-------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) for air_temperature_1 on test data = 0.008902094453156347
+------------------+-------------------+
| air_temperature_1|         prediction|
+------------------+-------------------+
|0.3558696988322065|0.35740081458738404|
|0.4394591272280271| 0.4358974670041313|
| 0.357918459332104| 0.3575136488084082|
|0.4357713583282114| 0.4283559566373961|
|0.3614013521819299| 0.3581713865841553|
+------------------+-------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) for rel

Main function to train the predict the values using ensembling method and using air_temperature_1 as target variable.

In [ ]:
import random
def train_and_evaluate_model(df, all_features, target, num_features):
    selected_features = random.sample(all_features, num_features)
    assembler = VectorAssembler(inputCols=selected_features, outputCol="features")
    rf = RandomForestRegressor(featuresCol="features", labelCol=target, numTrees=50, maxDepth=10)
    pipeline = Pipeline(stages=[assembler, rf])
    train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)
    model = pipeline.fit(train_data)
    predictions = model.transform(test_data)

    # Evaluating the model using the RMSE metric
    evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE) for {target} with features {selected_features} = {rmse}")

    return model, predictions,rmse

feature_columns = [
    'code_1', 'air_temperature_1', 'relative_humidity_1', 'precipitation_1',
    'wind_speed_1', 'wind_direction_1', 'air_pressure_2', 'air_temperature_2',
    'relative_humidity_2', 'precipitation_2', 'wind_speed_2', 'wind_direction_2',
    'hour_2', 'day_of_week_2', 'month_2', 'hour_1', 'day_of_week_1', 'month_1'
]
target = 'air_temperature_1'

In [ ]:
num_models = 5  # Number of models in the ensemble
num_features = 10  # Number of features to select randomly

models = []
predictions_list = []
rmse_values=[]

for _ in range(num_models):
    model, predictions,rmse = train_and_evaluate_model(df, feature_columns, target, num_features)
    models.append(model)
    predictions_list.append(predictions.withColumnRenamed("prediction", f"prediction_{_}"))
    rmse_values.append(rmse)
average_rmse=sum(rmse_values)/len(rmse_values)
print('The average rmse of ensemble model is:',average_rmse)

Root Mean Squared Error (RMSE) for air_temperature_1 with features ['hour_2', 'hour_1', 'month_2', 'air_temperature_1', 'air_temperature_2', 'precipitation_1', 'relative_humidity_1', 'day_of_week_1', 'code_1', 'air_pressure_2'] = 0.00804373128970814
Root Mean Squared Error (RMSE) for air_temperature_1 with features ['code_1', 'hour_2', 'hour_1', 'wind_speed_1', 'precipitation_1', 'wind_direction_1', 'air_pressure_2', 'month_1', 'precipitation_2', 'relative_humidity_2'] = 0.05718314529111583
Root Mean Squared Error (RMSE) for air_temperature_1 with features ['hour_2', 'precipitation_1', 'day_of_week_2', 'wind_direction_1', 'relative_humidity_2', 'day_of_week_1', 'code_1', 'air_temperature_1', 'precipitation_2', 'wind_direction_2'] = 0.012486231234722488
Root Mean Squared Error (RMSE) for air_temperature_1 with features ['air_temperature_1', 'air_pressure_2', 'air_temperature_2', 'wind_speed_2', 'hour_2', 'wind_direction_1', 'month_1', 'wind_direction_2', 'wind_speed_1', 'day_of_week_2']

The model is trained by running random forest classifier over a loop of 5 with 10 random features extracted in each run and accounting for less rmse in each run.The total average rmse is 0.02